In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


bs013 = pd.merge(pd.read_csv('data/BS013_trans.csv', sep=';', decimal=',', ), pd.read_csv('data/BS013_refl.csv', sep=';', decimal=',', ), how='outer', on='Wavelength (nm)', suffixes=('trans', 'refl')).dropna(how='all', axis=1)
bs025 = pd.merge(pd.read_csv('data/BS025_trans.csv', sep=';', decimal=',', ), pd.read_csv('data/BS025_refl.csv', sep=';', decimal=',', ), how='outer', on='Wavelength (nm)', suffixes=('trans', 'refl')).dropna(how='all', axis=1)
dmlp463 = pd.read_csv('data/DMLP463.csv', sep=';', decimal=',')
dmlp550 = pd.read_csv('data/DMLP550.csv', sep=';', decimal=',')
dmlp605 = pd.read_csv('data/DMLP605.csv', sep=';', decimal=',')
fbh450 = pd.read_csv('data/FBH450-10.csv', sep=';', decimal=',')
fbh480 = pd.read_csv('data/FBH480-10.csv', sep=';', decimal=',')
fbh515 = pd.read_csv('data/FBH515_trans.csv', sep=';', decimal=',')
fbh570 = pd.read_csv('data/FBH570-10.csv', sep=';', decimal=',')
fbh590 = pd.read_csv('data/FBH590-10.csv', sep=';', decimal=',')
fbh780 = pd.read_csv('data/FBH780-10.csv', sep=';', decimal=',')

In [ ]:
tables = {
  # "bs013": bs013,
  "bs025": bs025,
  "dmlp463": dmlp463,
  "dmlp550": dmlp550,
  "dmlp605": dmlp605,
  "fbh450": fbh450,
  "fbh480": fbh480,
  "fbh515": fbh515,
  "fbh570": fbh570,
  "fbh590": fbh590,
  "fbh780": fbh780,
}

# Convert bs013 columns to numeric and normalize
spec = bs013.add_prefix('bs013_')
spec.rename(columns={'bs013_Wavelength (nm)': 'Wavelength (nm)'}, inplace=True)
spec.loc[:, spec.columns != "Wavelength (nm)"] = spec.loc[:, spec.columns != "Wavelength (nm)"]/100

# Process each table
for name, table in tables.items():
  # Rename columns that start with '%'
  for col in table.columns:
    if col.startswith('%'):
      table.rename(columns={col: col.replace('% ', '')}, inplace=True)
  
  # Convert all non-wavelength columns to numeric first
  for col in table.columns:
    if col != 'Wavelength (nm)':
      table[col] = pd.to_numeric(table[col], errors='coerce')
  
  # Divide by 100 to normalize
  table.loc[:, table.columns != "Wavelength (nm)"] = table.loc[:, table.columns != "Wavelength (nm)"] / 100
  
  # Add prefix and merge
  table = table.add_prefix(f'{name}_')
  table.rename(columns={f'{name}_Wavelength (nm)': 'Wavelength (nm)'}, inplace=True)
  spec = spec.merge(table, how='outer', on='Wavelength (nm)')

# spec['dmlp550_% Reflectance'] = spec['dmlp550_% Reflectance'].apply(lambda x: float(x))
# spec['dmlp550_% Transmission'] = spec['dmlp550_% Transmission'].apply(lambda x: float(x))


# spec.set_index("Wavelength (nm)", inplace=True)
spec.interpolate(method='linear', inplace=True)

In [ ]:
# Plot the transmission data
fig, axes = plt.subplots(7, 2, figsize=(24, 42))

wholePP = spec['bs025_P-Polarizedrefl']

wholeSP = spec['bs025_S-Polarizedrefl']

O777PP = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Transmission'] * spec['dmlp605_Transmission'] * spec['bs013_P-Polarizedrefl'] * spec['fbh780_Transmission']
# Ha656PP = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Transmission'] * spec['dmlp605_Transmission'] * spec['bs013_P-Polarizedtrans'] * spec['fbh780'] waiting for data
HeI588 = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Transmission'] * spec['dmlp605_Reflectance'] * spec['bs013_P-Polarizedrefl'] * spec['fbh590_Transmission']
NII569 = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Transmission'] * spec['dmlp605_Reflectance'] * spec['bs013_P-Polarizedtrans'] * spec['fbh570_Transmission']
HeI447 = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Reflectance'] * spec['dmlp463_Reflectance'] * spec['fbh450_Transmission']
ClII479 = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Reflectance'] * spec['dmlp463_Transmission'] * spec['bs013_P-Polarizedrefl'] * spec['fbh480_Transmission']
CII515 = spec['bs025_P-Polarizedtrans'] * spec['dmlp550_Reflectance'] * spec['dmlp463_Transmission'] * spec['bs013_P-Polarizedtrans'] * spec['fbh515_Transmission']

# Plot individual transmission components
axes[0][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[0][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Transmission'], 
         label='DMLP550 Trans', linewidth=2, alpha=0.7)
axes[0][0].plot(spec['Wavelength (nm)'], spec['dmlp605_Transmission'], 
         label='DMLP605 Trans', linewidth=2, alpha=0.7)
axes[0][0].plot(spec['Wavelength (nm)'], spec['bs013_P-Polarizedrefl'], 
         label='BS013 P-Polarized Refl', linewidth=2, alpha=0.7)
axes[0][0].plot(spec['Wavelength (nm)'], spec['fbh780_Transmission'], 
         label='FBH780 Trans', linewidth=2, alpha=0.7)
axes[0][0].set_xlabel('Wavelength (nm)')
axes[0][0].set_ylabel('Transmission/Reflection (%)')
axes[0][0].set_title('Individual Transmission Components for O777PP')
axes[0][0].legend()
axes[0][0].grid(True, alpha=0.3)

# Plot combined O777PP transmission
axes[0][1].plot(spec['Wavelength (nm)'], O777PP, 
         label='O777PP (Combined)', linewidth=2, color='red')
axes[0][1].axvline(x=777.5, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 777.5 nm')
axes[0][1].set_xlabel('Wavelength (nm)')
axes[0][1].set_ylabel('Combined Transmission (%)')
axes[0][1].set_title('O777PP Combined Transmission')
axes[0][1].legend()
axes[0][1].grid(True, alpha=0.3)

# Plot individual transmission components
axes[2][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[2][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Transmission'], 
         label='DMLP550 Trans', linewidth=2, alpha=0.7)
axes[2][0].plot(spec['Wavelength (nm)'], spec['dmlp605_Reflectance'], 
         label='DMLP605 Refl', linewidth=2, alpha=0.7)
axes[2][0].plot(spec['Wavelength (nm)'], spec['bs013_P-Polarizedrefl'], 
         label='BS013 P-Polarized Refl', linewidth=2, alpha=0.7)
axes[2][0].plot(spec['Wavelength (nm)'], spec['fbh590_Transmission'], 
         label='FBH590 Trans', linewidth=2, alpha=0.7)
axes[2][0].set_xlabel('Wavelength (nm)')
axes[2][0].set_ylabel('Transmission/Reflection (%)')
axes[2][0].set_title('Individual Transmission Components for HeI588')
axes[2][0].legend()
axes[2][0].grid(True, alpha=0.3)

# Plot combined HeI588 transmission
axes[2][1].plot(spec['Wavelength (nm)'], HeI588, 
         label='HeI588 (Combined)', linewidth=2, color='blue')
axes[2][1].axvline(x=587.6, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 587.6 nm')
axes[2][1].set_xlabel('Wavelength (nm)')
axes[2][1].set_ylabel('Combined Transmission (%)')
axes[2][1].set_title('HeI588 Combined Transmission')
axes[2][1].legend()
axes[2][1].grid(True, alpha=0.3)

# Plot individual transmission components for NII569
axes[3][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[3][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Transmission'], 
         label='DMLP550 Trans', linewidth=2, alpha=0.7)
axes[3][0].plot(spec['Wavelength (nm)'], spec['dmlp605_Reflectance'], 
         label='DMLP605 Refl', linewidth=2, alpha=0.7)
axes[3][0].plot(spec['Wavelength (nm)'], spec['bs013_P-Polarizedtrans'], 
         label='BS013 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[3][0].plot(spec['Wavelength (nm)'], spec['fbh570_Transmission'], 
         label='FBH570 Trans', linewidth=2, alpha=0.7)
axes[3][0].set_xlabel('Wavelength (nm)')
axes[3][0].set_ylabel('Transmission/Reflection (%)')
axes[3][0].set_title('Individual Transmission Components for NII569')
axes[3][0].legend()
axes[3][0].grid(True, alpha=0.3)

# Plot combined NII569 transmission
axes[3][1].plot(spec['Wavelength (nm)'], NII569, 
         label='NII569 (Combined)', linewidth=2, color='orange')
axes[3][1].axvline(x=568.6, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 568.6 nm')
axes[3][1].set_xlabel('Wavelength (nm)')
axes[3][1].set_ylabel('Combined Transmission (%)')
axes[3][1].set_title('NII569 Combined Transmission')
axes[3][1].legend()
axes[3][1].grid(True, alpha=0.3)

# Plot individual transmission components for HeI447
axes[4][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[4][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Reflectance'], 
         label='DMLP550 Refl', linewidth=2, alpha=0.7)
axes[4][0].plot(spec['Wavelength (nm)'], spec['dmlp463_Reflectance'], 
         label='DMLP463 Refl', linewidth=2, alpha=0.7)
axes[4][0].plot(spec['Wavelength (nm)'], spec['fbh450_Transmission'], 
         label='FBH450 Trans', linewidth=2, alpha=0.7)
axes[4][0].set_xlabel('Wavelength (nm)')
axes[4][0].set_ylabel('Transmission/Reflection (%)')
axes[4][0].set_title('Individual Transmission Components for HeI447')
axes[4][0].legend()
axes[4][0].grid(True, alpha=0.3)

# Plot combined HeI447 transmission
axes[4][1].plot(spec['Wavelength (nm)'], HeI447, 
         label='HeI447 (Combined)', linewidth=2, color='purple')
axes[4][1].axvline(x=447.1, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 447.1 nm')
axes[4][1].set_xlabel('Wavelength (nm)')
axes[4][1].set_ylabel('Combined Transmission (%)')
axes[4][1].set_title('HeI447 Combined Transmission')
axes[4][1].legend()
axes[4][1].grid(True, alpha=0.3)

# Plot individual transmission components for ClII479
axes[5][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[5][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Transmission'], 
         label='DMLP550 Trans', linewidth=2, alpha=0.7)
axes[5][0].plot(spec['Wavelength (nm)'], spec['dmlp463_Transmission'], 
         label='DMLP463 Trans', linewidth=2, alpha=0.7)
axes[5][0].plot(spec['Wavelength (nm)'], spec['bs013_P-Polarizedrefl'], 
         label='BS013 P-Polarized Refl', linewidth=2, alpha=0.7)
axes[5][0].plot(spec['Wavelength (nm)'], spec['fbh480_Transmission'], 
         label='FBH480 Trans', linewidth=2, alpha=0.7)
axes[5][0].set_xlabel('Wavelength (nm)')
axes[5][0].set_ylabel('Transmission/Reflection (%)')
axes[5][0].set_title('Individual Transmission Components for ClII479')
axes[5][0].legend()
axes[5][0].grid(True, alpha=0.3)

# Plot combined ClII479 transmission
axes[5][1].plot(spec['Wavelength (nm)'], ClII479, 
         label='ClII479 (Combined)', linewidth=2, color='green')
axes[5][1].axvline(x=479.5, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 479.5 nm')
axes[5][1].set_xlabel('Wavelength (nm)')
axes[5][1].set_ylabel('Combined Transmission (%)')
axes[5][1].set_title('ClII479 Combined Transmission')
axes[5][1].legend()
axes[5][1].grid(True, alpha=0.3)

# Plot individual transmission components for CII515
axes[6][0].plot(spec['Wavelength (nm)'], spec['bs025_P-Polarizedtrans'], 
         label='BS025 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[6][0].plot(spec['Wavelength (nm)'], spec['dmlp550_Transmission'], 
         label='DMLP550 Trans', linewidth=2, alpha=0.7)
axes[6][0].plot(spec['Wavelength (nm)'], spec['dmlp463_Transmission'], 
         label='DMLP463 Trans', linewidth=2, alpha=0.7)
axes[6][0].plot(spec['Wavelength (nm)'], spec['bs013_P-Polarizedtrans'], 
         label='BS013 P-Polarized Trans', linewidth=2, alpha=0.7)
axes[6][0].plot(spec['Wavelength (nm)'], spec['fbh515_Transmission'], 
         label='FBH515 Trans', linewidth=2, alpha=0.7)
axes[6][0].set_xlabel('Wavelength (nm)')
axes[6][0].set_ylabel('Transmission/Reflection (%)')
axes[6][0].set_title('Individual Transmission Components for CII515')
axes[6][0].legend()
axes[6][0].grid(True, alpha=0.3)

# Plot combined CII515 transmission
axes[6][1].plot(spec['Wavelength (nm)'], CII515, 
         label='CII515 (Combined)', linewidth=2, color='cyan')
axes[6][1].axvline(x=514.5, color='black', linestyle='--', linewidth=1.5, alpha=0.7, label='Target: 514.5 nm')
axes[6][1].set_xlabel('Wavelength (nm)')
axes[6][1].set_ylabel('Combined Transmission (%)')
axes[6][1].set_title('CII515 Combined Transmission')
axes[6][1].legend()
axes[6][1].grid(True, alpha=0.3)

# Set x-axis limits for all subplots
for ax_row in axes:
    for ax in ax_row:
        ax.set_xlim(400, 850)

mpl.rcParams['figure.dpi'] = 400

plt.tight_layout()
plt.show()